In [2]:
cd ../

/nfs/home/dgranziol/kfac-curvature


In [3]:
import argparse
import time
import tabulate

import numpy as np
import matplotlib.pyplot as plt

from gpytorch.utils.lanczos import lanczos_tridiag

import torch

from curvature import data, models, losses, utils
from curvature.methods.swag import SWAG
import scipy.io as sio

Older torchvision found


/nfs/home/dgranziol/anaconda2/envs/newtensor/lib/python3.7/site-packages/torchvision/transforms/transforms.py:396: UserWarning: The use of the transforms.RandomSizedCrop transform is deprecated, please use transforms.RandomResizedCrop instead.
  warnings.warn("The use of the transforms.RandomSizedCrop transform is deprecated, " +


In [4]:
dataset = 'CIFAR100'
data_path = '/nfs/home/dgranziol/kfac-curvature/data'
#LOGISTIC REGRESSION MNIST
ckpt = '/nfs/home/dgranziol/kfac-curvature/out/CIFAR100/VGG16/SGD/seed=1_lr=0.05_mom=0.9_wd=0.0005_batchsize=128_numepochs=300/checkpoint-00000.pt'

dataset = ckpt.split('/out/')[1].split('/')[0]
model = ckpt.split('/seed=')[0].split('/')[7]
network = model
epochsave = int(ckpt.split('checkpoint-')[1].split('.')[0])

# #MLP small MNIST
# ckpt = '/nfs/home/dgranziol/kfac-curvature/out/MNIST/MLP_sdp/SGD/seed=1_lr=0.03_mom=0.9_wd=0.0_batchsize=128_numepochs=200/checkpoint-00200.pt'


#model = 'Logistic'
print('Using model %s' % model)
#from curvature.models import logistic_regression
model_cfg = getattr(models,model)

Using model VGG16


In [5]:
batch_size = 256

datasets, num_classes = data.datasets(
    dataset,
    data_path,
    transform_train=model_cfg.transform_test,
    transform_test=model_cfg.transform_test,
    use_validation=False,
    train_subset=None,
    train_subset_seed=None,
)
loader = torch.utils.data.DataLoader(
    datasets['train'],
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)
# full_datasets, _ = data.datasets(
#     dataset,
#     data_path,
#     transform_train=model_cfg.transform_train,
#     transform_test=model_cfg.transform_test,
#     use_validation=False,
# )
# full_loader = torch.utils.data.DataLoader(
#     full_datasets['train'],
#     batch_size=batch_size,
#     shuffle=False,
#     num_workers=4,
#     pin_memory=True
# )

Loading CIFAR100 from /nfs/home/dgranziol/kfac-curvature/data
Files already downloaded and verified
You are going to run models on the test set. Are you sure?
Files already downloaded and verified
Using train (50000) + test (10000)


In [6]:
model = model_cfg.base(*model_cfg.args, num_classes=num_classes, **model_cfg.kwargs)
print('Loading %s' % ckpt)
checkpoint = torch.load(ckpt)
model.load_state_dict(checkpoint['state_dict'])

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
#device = torch.device('cpu')
model.to(device)
num_parametrs = sum([p.numel() for p in model.parameters()])
criterion = losses.cross_entropy


Loading /nfs/home/dgranziol/kfac-curvature/out/CIFAR100/VGG16/SGD/seed=1_lr=0.05_mom=0.9_wd=0.0005_batchsize=128_numepochs=300/checkpoint-00000.pt


In [7]:
#Lanczos if you want it

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [15]:
param_list = list(model.parameters())
vector_list = []
vector2_list = []
vector = torch.randn(num_parametrs)
vector2 = torch.zeros(num_parametrs)
vector = vector/torch.dot(vector,vector)

offset = 0
for param in param_list:
    vector_list.append(vector[offset:offset + param.numel()].detach().view_as(param).to(param.device))
    vector2_list.append(vector2[offset:offset + param.numel()].detach().view_as(param).to(param.device))
    offset += param.numel()

model.eval()
#     if bn_train_mode:
#         model.apply(_bn_train_mode)

model.zero_grad()
N = len(loader.dataset)
for input, target in loader:
    input = input.cuda(non_blocking=True)
    target = target.cuda(non_blocking=True)
    loss, _, _ = criterion(model, input, target)
    loss *= input.size()[0] / N
    #poo = [torch.sign(p) for p in param_list]
    grad_list = torch.autograd.grad(loss, param_list, create_graph=True)
    grad_list2 = np.abs(param_list)*grad_list
    dL_dvec = torch.zeros(1)
    dL_dvec = dL_dvec.cuda()
    for v, g in zip(vector_list, grad_list2):
        dL_dvec += torch.sum(v * g)
    dL_dvec.backward()
    #vector2_list = torch.cat((vector2_list,torch.autograd.grad(dL_dvec, param_list, create_graph=False)),0)
    #print(param_list[0].grad.size())
model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [16]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
np.abs(param_list)

In [ ]:
dL_dvec = torch.zeros(1)
dL_dvec = dL_dvec.cuda()
for v, g in zip(vector_list, grad_list2):
    dL_dvec += torch.sum(v * g)
dL_dvec.backward(retain_graph=True)

In [ ]:
grad_list

In [ ]:
param_list

In [ ]:
np.abs(param_list)*grad_list
torch.mul([abs(p) for p in param_list],grad_list)

In [ ]:
w = torch.cat([param.detach().cpu().view(-1) for param in model.parameters()])
w_l2_norm = torch.norm(w).numpy()
w_linf_norm = torch.norm(w, float('inf')).numpy()

In [ ]:
def hess_vec(vector, loader, model, criterion, cuda=True, bn_train_mode=True):
    param_list = list(model.parameters())
    vector_list = []

    offset = 0
    for param in param_list:
        vector_list.append(vector[offset:offset + param.numel()].detach().view_as(param).to(param.device))
        offset += param.numel()

    model.eval()
#     if bn_train_mode:
#         model.apply(_bn_train_mode)

    model.zero_grad()
    N = len(loader.dataset)
    for input, target in loader:
        input = input.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)
        loss, _, _ = criterion(model, input, target)
        loss *= input.size()[0] / N

        grad_list = torch.autograd.grad(loss, param_list, create_graph=True)
        dL_dvec = torch.zeros(1)
        if cuda:
            dL_dvec = dL_dvec.cuda()
        for v, g in zip(vector_list, grad_list):
            dL_dvec += torch.sum(v * g)
        dL_dvec.backward()
        #print(param_list[0].grad.size())
    model.eval()

In [ ]:
class CurvVecProduct(object):
    def __init__(self, loader, model, criterion, curvature_matrix, full_loader=None):
        self.loader = loader
        self.full_loader = full_loader
        self.model = model
        self.criterion = criterion
        self.iters = 0
        self.timestamp = time.time()
        self.curvature_matrix = curvature_matrix

    def __call__(self, vector):
        start_time = time.time()
        if self.curvature_matrix == 'hessian':
            output = hess_vec(
                vector,
                self.loader,
                self.model,
                self.criterion,
                cuda='cuda'
               
                )
        elif self.curvature_matrix == 'covgrad':
            output = utils.covgrad_vec(
                vector,
                self.loader,
                self.model,
                self.criterion,
                cuda='cuda',
            )
        else:
            raise ValueError("Unrecognised curvature_matrix argument "+self.curvature_matrix)
        time_diff = time.time() - start_time

        self.iters += 1
        print('Iter %d. Time: %.2f' % (self.iters, time_diff))
        # return output.unsqueeze(1)¬
        return output.cpu().unsqueeze(1)

w = torch.cat([param.detach().cpu().view(-1) for param in model.parameters()])
w_l2_norm = torch.norm(w).numpy()
w_linf_norm = torch.norm(w, float('inf')).numpy()

In [ ]:
#Hessian
productor = CurvVecProduct(loader, model, criterion, 'hessian')
#utils.bn_update(loader, model)
Q, T = lanczos_tridiag(productor, 3, dtype=torch.float32, device='cpu', matrix_shape=(num_parametrs, num_parametrs))

eigvals, eigvects = T.eig(eigenvectors=True)
gammas = eigvects[0, :] ** 2
V = eigvects.t() @ Q.t()

In [ ]:
eigval = []
for i in range(0,len(eigvals)):
    eigval.append(eigvals[i][0])
mean = np.dot(eigval,gammas)

In [ ]:
#For MacKay
alpha = 1e-5 
gamm = num_parametrs(1-np.dot((alpha/eigvals+alpha),gammas))
alphanew = gamm/torch.norm(w).numpy()